## **Procesar el Json de `fichas_tecnicas_mapped.json` para poder trabajar con Chatbots**

#### **Import Libraries**

In [5]:
import json

#### **Load Data**

In [ ]:
# Ruta del archivo JSON (ajusta la ruta si es necesario)
json_path = "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json"

# Cargar el JSON
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Mostrar los primeros elementos para revisar la estructura
print(
    json.dumps(data[:2], indent=4, ensure_ascii=False)
)  # Mostramos solo 2 para no saturar

[
    {
        "indicaciones": "en base a su efecto antiagregante plaquetario está indicado en la profilaxis de: infarto de miocardio o reinfarto de miocardio en pacientes con angina de pecho inestable y para prevenir la recurrencia del mismo en pacientes con historia de infarto de miocardio. prevención de la oclusión del bypass aortocoronario. tromboflebitis, flebotrombosis y riesgo de trombosis arteriales. tromboembolismo post-operatorio en pacientes con prótesis vasculares biológicas o con shunts arteriovenosos. tratamiento de ataques isquémicos transitorios en varones con isquemia cerebral transitoria para reducir el riesgo de accidente cerebrovascular. estas indicaciones requieren siempre un control médico.",
        "posologia": "posología: como inhibidor de la agregación plaquetaria: en la prevención de accidentes tromboembólicos (infarto, angina de pecho, prótesis valvulares biológicas, by pass arteriales, tromboflebitis, flebotrombosis y riesgo de trombosis arterial). dosis m

In [7]:
# Mostrar todas las claves de un solo medicamento para entender la estructura
print("Claves disponibles en el JSON:")
print(list(data[0].keys()))

Claves disponibles en el JSON:
['indicaciones', 'posologia', 'contraindicaciones', 'advertencias', 'interacciones', 'fertilidad_embarazo', 'efectos_conducir', 'reacciones_adversas', 'sobredosis', 'ATC', 'Propiedades_farmacocineticas', 'excipientes', 'incompatibilidades', 'precauciones_conservacion', 'fecha_revision', 'nombre_archivo', 'ATC_Nivel_Anatomico', 'Descripcion_Nivel_Anatomico', 'ATC_Nivel_2_Subgrupo_Terapeutico', 'Descripcion_Nivel_2_Subgrupo_Terapeutico', 'ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico', 'Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico', 'ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico', 'Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico', 'ATC_Nivel_5_Principio_Activo', 'Descripcion_Nivel_5_Principio_Activo']


#### **Convertir el JSON en fragmentos**

Queremos convertir el JSON a una estructura tal que así:

```python
[
    {
        "medicamento": "Ácido acetilsalicílico",
        "categoria": "indicaciones",
        "texto": "En base a su efecto antiagregante plaquetario está indicado en..."
    },
    {
        "medicamento": "Ácido acetilsalicílico",
        "categoria": "advertencias",
        "texto": "Dado el efecto antiagregante plaquetario del ácido acetilsalicílico..."
    },
    ...
]
```

##### **Verificar valores `None` en los campos**

In [ ]:
# Recorremos algunos medicamentos y verificamos si tienen valores None
for medicamento in data[:5]:  # Solo mostramos los primeros 5
    for campo in campos_interes:
        valor = medicamento.get(campo, "No encontrado")
        print(
            f"Medicamento: {medicamento.get('nombre_archivo', 'Desconocido')}, Campo: {campo}, Tipo: {type(valor)}"
        )

Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: indicaciones, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: posologia, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: contraindicaciones, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: advertencias, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: interacciones, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: fertilidad_embarazo, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: efectos_conducir, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: reacciones_adversas, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: sobredosis, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: Propiedades_farmacocineticas, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt, Campo: excipientes, Tipo: <class 'str'>
Medicamento: A.A.S._100_mg_COM

##### **Confirmar si hay `None` en todo el dataset**

In [ ]:
# Buscamos si hay valores None en los campos de interés
for medicamento in data:
    for campo in campos_interes:
        if medicamento.get(campo) is None:
            print(
                f"⚠️ {medicamento.get('nombre_archivo', 'Desconocido')} tiene {campo} como None"
            )

⚠️ ABIOX_400_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA_EFG.txt tiene Propiedades_farmacocineticas como None
⚠️ ABIOX_400_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA_EFG.txt tiene excipientes como None
⚠️ ABIOX_400_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA_EFG.txt tiene incompatibilidades como None
⚠️ ABIOX_400_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA_EFG.txt tiene precauciones_conservacion como None
⚠️ ACCOFIL_12_MU_0,2_ML_SOLUCION_INYECTABLE_Y_PARA_PERFUSION_EN_JERINGA_PRECARGADA.txt tiene Propiedades_farmacocineticas como None
⚠️ ACCOFIL_12_MU_0,2_ML_SOLUCION_INYECTABLE_Y_PARA_PERFUSION_EN_JERINGA_PRECARGADA.txt tiene excipientes como None
⚠️ ACCOFIL_12_MU_0,2_ML_SOLUCION_INYECTABLE_Y_PARA_PERFUSION_EN_JERINGA_PRECARGADA.txt tiene incompatibilidades como None
⚠️ ACCOFIL_12_MU_0,2_ML_SOLUCION_INYECTABLE_Y_PARA_PERFUSION_EN_JERINGA_PRECARGADA.txt tiene precauciones_conservacion como None
⚠️ ACICLOVIR_SALA_250_MG_POLVO_PARA_SOLUCION_PARA_PERFUSION_EFG.txt tiene excipientes como None
⚠️ ACICLOVIR